In [8]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [2]:
DATA_PATH = "dataset/dickson_liver_cirrhosis.csv"
TARGET_COL = "Stage"

In [3]:
data_df = pd.read_csv(DATA_PATH)
data_df.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,2221,C,Placebo,18499,F,N,Y,N,N,0.5,149.0,4.04,227.0,598.0,52.70,57.0,256.0,9.9,1
1,1230,C,Placebo,19724,M,Y,N,Y,N,0.5,219.0,3.93,22.0,663.0,45.00,75.0,220.0,10.8,2
2,4184,C,Placebo,11839,F,N,N,N,N,0.5,320.0,3.54,51.0,1243.0,122.45,80.0,225.0,10.0,2
3,2090,D,Placebo,16467,F,N,N,N,N,0.7,255.0,3.74,23.0,1024.0,77.50,58.0,151.0,10.2,2
4,2105,D,Placebo,21699,F,N,Y,N,N,1.9,486.0,3.54,74.0,1052.0,108.50,109.0,151.0,11.5,1


In [5]:
object_cols=[col for col in data_df.columns if data_df[col].dtype=="object"]
number_of_unique={col:len(data_df[col].unique()) for col in object_cols}

print(object_cols)
print(number_of_unique)

['Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
{'Status': 3, 'Drug': 2, 'Sex': 2, 'Ascites': 2, 'Hepatomegaly': 2, 'Spiders': 2, 'Edema': 3}


In [6]:
ohe = OneHotEncoder()
ord = OrdinalEncoder()

cols_for_ordinal = ["Status", "Sex"]
cols_for_ohe = ["Drug", "Ascites", "Hepatomegaly", "Spiders", "Edema"]

ord_result  = ord.fit_transform(data_df[cols_for_ordinal])
ohe_result = ohe.fit_transform(data_df[cols_for_ohe])

columns_ohe = ohe.get_feature_names_out(cols_for_ohe)

ord_df = pd.DataFrame(ord_result, columns=cols_for_ordinal)
ohe_df = pd.DataFrame(ohe_result.toarray(), columns=columns_ohe)

data_df = pd.concat([data_df, ohe_df], axis=1).drop(columns=cols_for_ohe)
data_df[cols_for_ordinal] = ord_df

In [7]:
train_x, train_y = data_df.drop(TARGET_COL, axis=1).values, data_df[TARGET_COL].values

In [9]:
class TabDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.targets[idx]
        return x, y

In [10]:
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).long()

num_class = len(np.unique(train_y))
print(f"Number of classes: {num_class}")

Number of classes: 3


In [11]:
dataset = TabDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [16]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x):
        # Ensure x has three dimensions: (batch_size, sequence_length, features)
        if x.dim() < 3:
            x = x.unsqueeze(1)  # Example adjustment, depends on expected input shape

        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)

        # Ensure keys are transposed correctly for batch matrix multiplication
        keys_transposed = keys.transpose(1, 2)

        # Perform batch matrix multiplication
        scores = torch.bmm(queries, keys_transposed) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        return weighted

In [17]:
class AttentiveMLP(nn.Module):
  def __init__(self, input_dim, num_class):
    super(AttentiveMLP, self).__init__()
    self.fc1 = nn.Linear(input_dim, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 128),
    self.fc4 = nn.Linear(128, num_class)

    self.att1 = SelfAttention(512)
    self.att2 = SelfAttention(128)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.att1(x)
    x = F.relu(self.fc2(x))
    x = self.att2(x)
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    x = F.softmax(x, dim=1)
    return x, x

In [18]:
input_size = 24

model = AttentiveMLP(input_size, num_class)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloader:
        optimizer.zero_grad()

        outputs, _ = model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')